In [ ]:
import pandas as pd
import numpy as np 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


In [5]:
nltk.download('vader_lexicon')


2.7


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\msouh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [30]:
analyzer=SentimentIntensityAnalyzer()
vaderLex=analyzer.lexicon
print(vaderLex.get("love"))#Does not detect with CAPITAL letters

3.2


In [33]:
print(analyzer.polarity_scores("love you so much , see you bye , i hate you"))

{'neg': 0.248, 'neu': 0.47, 'pos': 0.282, 'compound': 0.128}


In [ ]:
training_set=pd.read_csv("train.csv")

In [16]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1985 entries, 0 to 1984
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   desc_id           1985 non-null   int64 
 1   source_name       1985 non-null   object
 2   review_text       1984 non-null   object
 3   is_accessibility  1985 non-null   object
 4   cleanText         1982 non-null   object
 5   Vader Sentiment   1985 non-null   object
dtypes: int64(1), object(5)
memory usage: 93.2+ KB


### No nan Values

In [17]:
training_set.head()

,desc_id,source_name,review_text,is_accessibility,cleanText,Vader Sentiment
0,1,gp_AOqpTOE-4PtpSwSKzv8CF6EtR6seFSLwg-QQPTpQKze...,Excellent app. Been using it for years on man...,Accessibility Review,excellent app using years many different phone...,Positive
1,1,gp_AOqpTOGPKJI_1QuwFgSTcvd7BanaHdhJoLUXZDx8OVt...,I've used this on a different phone - but cur...,Accessibility Review,used different phone currently moto android re...,Positive
2,1,lg_AOqpTOE5_m1p9SEdAbCNs_Nh_qN7EHMLnyPnytEW9NX...,Fantastic program! Instant search results and...,Accessibility Review,fantastic program instant search results fast ...,Positive
3,1,lg_AOqpTOEyZAX-Z1qAqVvbqV8s6vJAttZvFBFixQTM_jW...,I'm using this on my mobile as well as on PC ...,Accessibility Review,using mobile well view offline dictionaries us...,Positive
4,1,gp_AOqpTOE6SfnT_RCAb9w8AbMpASAbJDvsJIHxp_xuhKg...,Text reflow doesn't work. I want my money back...,Accessibility Review,text reflow work want money back,Positive


In [19]:
training_set[training_set['Vader Sentiment']=="Neutral"].count()

desc_id             181
source_name         181
review_text         180
is_accessibility    181
cleanText           178
Vader Sentiment     181
dtype: int64

In [ ]:
training_set = training_set.drop(index=training_set[training_set['Vader Sentiment'] == "Neutral"].index)#Dropping neutral sentiment


In [26]:
training_set[training_set['Vader Sentiment']=="Neutral"].count()

desc_id             0
source_name         0
review_text         0
is_accessibility    0
cleanText           0
Vader Sentiment     0
dtype: int64

In [55]:
sentence="excellent app using years many different phones automatically read text messages though cars bluetooth voice stream wish could direct google maps voice commands voice stream well hopefully day"

list=[]
for word in sentence.split(sep=" "):
    score=vaderLex.get(word)
    print("Word :",word,"\tScore : ",vaderLex.get(word))
    list.append(score)

print(sentence)    
print(list)

Word : excellent 	Score :  2.7
Word : app 	Score :  None
Word : using 	Score :  None
Word : years 	Score :  None
Word : many 	Score :  None
Word : different 	Score :  None
Word : phones 	Score :  None
Word : automatically 	Score :  None
Word : read 	Score :  None
Word : text 	Score :  None
Word : messages 	Score :  None
Word : though 	Score :  None
Word : cars 	Score :  None
Word : bluetooth 	Score :  None
Word : voice 	Score :  None
Word : stream 	Score :  None
Word : wish 	Score :  1.7
Word : could 	Score :  None
Word : direct 	Score :  None
Word : google 	Score :  None
Word : maps 	Score :  None
Word : voice 	Score :  None
Word : commands 	Score :  None
Word : voice 	Score :  None
Word : stream 	Score :  None
Word : well 	Score :  1.1
Word : hopefully 	Score :  1.7
Word : day 	Score :  None
excellent app using years many different phones automatically read text messages though cars bluetooth voice stream wish could direct google maps voice commands voice stream well hopefully day
[2

In [56]:
list=[]
for each in training_set['cleanText']:
    list=[]
    for word in training_set['cleanText'].split(sep=" "):
        score=vaderLex.get(word)
        print("Word :",word,"\tScore : ",vaderLex.get(word))
        list.append(score)
    training_set['Numerical Features']= list

AttributeError: 'Series' object has no attribute 'split'

In [52]:
training_set.head(1)

,desc_id,source_name,review_text,is_accessibility,cleanText,Vader Sentiment
0,1,gp_AOqpTOE-4PtpSwSKzv8CF6EtR6seFSLwg-QQPTpQKze...,Excellent app. Been using it for years on man...,Accessibility Review,excellent app using years many different phone...,Positive


In [60]:
def compute_score(text):
    scores = []
    if isinstance(text, str):  
        for word in text.split():  
            score = vaderLex.get(word, 0)
            scores.append(score)  
    return scores  

[2.7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.7, 0, 0, 0, 0, 0, 0, 0, 0, 1.1, 1.7, 0]


In [ ]:
training_set['Numerical Features'] = training_set['cleanText'].apply(compute_score)#Applying the func for each row


In [63]:
training_set.tail(10)

,desc_id,source_name,review_text,is_accessibility,cleanText,Vader Sentiment,Numerical Features
1972,1,gp_AOqpTOG7eL7ggaBifMxAQYq3DONgXbSmE4xrSapWuSa...,Reader does not remember my previous settings....,Accessibility Review,reader remember previous settings must reset s...,Positive,"[0, 0, 0, 0, 0, 0, 0, 1.5, 0, 0, 0, 0, 0, 0, 0..."
1973,1,gp_AOqpTOG7HQ70T2meLjPlcgdLSLuSv93-lKRMnwhv89S...,This reader is free - supports a wide range o...,Accessibility Review,reader free supports wide range ebook formats ...,Positive,"[0, 2.3, 1.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1974,1,gp_AOqpTOGaiazwEEJXX8jwz6t6137-U_UVgyURrz0eno-...,I've tried several ebook apps. I think Aldiko...,Accessibility Review,tried several ebook apps think aldiko best pdf...,Positive,"[0, 0, 0, 0, 0, 0, 3.2, 0, 0, 1.3, 0, 3.1, 0, ..."
1975,1,gp_AOqpTOGaIBL9hjOsouH_n-tRQr_gJIocsDYSjY1e3mf...,So many settings. If you don't like something...,Accessibility Review,many settings like something probably change s...,Positive,"[0, 0, 1.5, 0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 0,..."
1976,1,gp_AOqpTOGegKdpY8TvkxpCKsZhKlhoS4TAC8SwwnDp7zp...,was excellent but it stopped to see books on ...,Accessibility Review,excellent stopped see books extcard directory ...,Positive,"[2.7, -0.9, 0, 0, 0, 0, 0, 0, 0, 0]"
1977,1,gp_AOqpTOGF94lKb8JB7Uhh04CbGKmZ5fVghsA3ctfEVcW...,More fonts will make it even better,Accessibility Review,fonts make even better,Positive,"[0, 0, 0, 1.9]"
1979,1,gp_AOqpTOGJZCmw6frfoemc77Yq4dp-1fxbK-puAcwE29Q...,By far the best app I've used for reading and...,Accessibility Review,far best app used reading quick easy simple ab...,Positive,"[0, 3.2, 0, 0, 0, 0, 1.9, 0, 0, 0, 0, 0, 0, 0,..."
1981,1,gp_AOqpTOGLFnORYm9M6DkAuG6-VJzlIUV0T4jPmGB7-gj...,it is realy nice - I loved it . it is top ca...,Accessibility Review,realy nice loved top candidate reader favorite...,Positive,"[0, 1.8, 2.9, 0.8, 0, 0, 2.0, 0, 0, 0, 0, 0, 0..."
1983,1,gp_AOqpTOGpOhiVI2WL_uwhHuu_ZhwYnTXwJLDPt-4-9_Z...,Slick layout and design and very functional b...,Accessibility Review,slick layout design functional sometimes throw...,Positive,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0, 0..."
1984,1,gp_AOqpTOGpwNMVoUce48nQ7_cVsgQ7bnZ3LB0bjnJ3qmF...,I loved the app and all features including th...,Accessibility Review,loved app features including flexibility layou...,Positive,"[2.9, 0, 0, 0, 1.4, 0, 1.5, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
X_New = np.array([np.ones(len(x)), x.flatten()]).T 
y = y.reshape(100, 1) 
num_iter = 1000
alpha = 0.01 
m = len(x)  
theta = np.zeros((2, 1),dtype = float) 
for i in range(num_iter): 
    gradients = X_New.T.dot(X_New.dot(theta)- y) #Backward Pass
    theta = theta - (1/m) * alpha * gradients #Updating the weights 